<a href="https://colab.research.google.com/github/Rosangelafl/exercicios_python_inovacao_civica/blob/main/desafio_m5_raspagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Raspagem da secretaria de educação de São Paulo

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install requests-html

In [13]:
!pip install folium==0.2.1

In [14]:
import csv
import pandas as pd
from requests_html import HTMLSession

In [17]:
url = "http://comprasnet.gov.br/acesso.asp?url=/livre/Pregao/lista_pregao_filtro.asp?Opc=2"

In [16]:
session = HTMLSession()
first_page = session.get(url)

Qual a URL você identificou nessa requisição que realiza a consulta? Guarde-a numa variável:

In [18]:
form_url = "http://comprasnet.gov.br/livre/Pregao/lista_pregao.asp"

In [19]:
formdata = {
    "Opc": "2",
    "rdTpPregao": "E",
    "lstSrp": "T",
    "lstICMS": "T",
    "lstSituacao": "5",
    "lstTipoSuspensao": "0",
    "co_uasg": "925013",
}

In [20]:
form_response = session.post(form_url, data=formdata)

In [32]:
form_response.text

'\n<html>\n<head>\n<meta http-equiv="expires" content="0">\n<meta http-equiv="cache-control" content="private">\n<meta http-equiv="pragma" content="no-cache">\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n<script language="javascript" type="text/javascript" SRC="../../imagens/action.js"></script>\n<link rel="stylesheet" href="../../imagens/style1.css" type="text/css">\n<script LANGUAGE="javascript">\n\tfunction Dados_Pregao(pCod, pTp){\n\t\tvar url = "dados_pregao.asp?prgcod="+pCod+"&Tipo="+pTp\n\t\twindow.open(url, \'DadosPregao\', \'maximized=yes, screenX=250, screenY=0, toolbar=no,location=no,directories=no,status=no,menubar=no,scrollbars=yes,resizable=no,copyhistory=no,width=550,height=400\');\n\t\treturn false;\n\t}\n\tfunction Motivo_Suspensao(pCod, pOpc){\n\t\tvar url = "motiv_suspensao.asp?prgCod="+pCod+"&Opc="+pOpc\n\t\twindow.open (url,\'Suspensao\',\'maximized=yes, screenX=250, screenY=0, toolbar=no,location=no,directories=no,status=no,menubar=no

Agora vamos raspar os itens da página
Analise a página de resultados para encontrar o Xpath que seleciona todas as linhas da tabela de resultados, ou seja, queremos que cada linha da tabela seja um resultado dessa nossa seleção.

Preencha aqui o Xpath correto para que obtenhamos os pregões da tabela de resultados (apenas as linhas que contém pregões mesmo!):

In [21]:
xpath = "//table[@class='td']//tr[@class='tex3']"
rows = form_response.html.xpath(xpath)

In [22]:
print(rows[0].html)

<tr bgcolor="#FFFFFF" class="tex3" valign="middle">
<td align="center">
<a href="#" onclick="lista_itens('962837','492021');">492021</a>
</td>
<td align="center">925013</td>
<td align="center">PMSP - SECRETARIA MUNICIPAL DE EDUCAÇÃO</td>
<td align="center">19/07/2021 08:00</td>
<td align="center">29/07/2021 09:30</td>
<td align="center">Julgamento/Habilitação</td>
<td align="center">
<br/>
<a href="javascript:void(0)" onclick="Esclar_Pregao(962837);">Esclarecimentos</a><br/>
<a href="javascript:void(0)" onclick="Avisos_Pregao(962837);">Avisos</a><br/>
</td>
</tr>


Queremos raspar o texto de cada campo apresentado na tabela (exceto o campo "Informações do Pregão") e salvar todos os itens em formato de dicionário numa lista de pregões

In [23]:
pregoes = []
for row in rows:
    numero_pregao = row.xpath("./tr/td[1]/a/text()")[0]
    codigo_uasg = row.xpath("./tr/td[2]/text()")[0]
    nome_uasg = row.xpath("./tr/td[3]/text()")[0]
    data_hora_inicio = row.xpath("./tr/td[4]/text()")[0]
    data_hora_abertura = row.xpath("./tr/td[5]/text()")[0]
    situacao = row.xpath("./tr/td[6]/text()")[0]
    item = {
        "numero_pregao": numero_pregao,
        "codigo_uasg": codigo_uasg,
        "nome_uasg": nome_uasg,
        "data_hora_inicio": data_hora_inicio,
        "data_hora_abertura": data_hora_abertura,
        "situacao": situacao,
    }
pregoes.append(item)

In [33]:
pregoes

[{'codigo_uasg': '925013',
  'data_hora_abertura': '21/09/2021 09:30',
  'data_hora_inicio': '09/09/2021 08:00',
  'nome_uasg': 'PMSP - SECRETARIA MUNICIPAL DE EDUCAÇÃO',
  'numero_pregao': '642021',
  'situacao': 'Julgamento/Habilitação'}]

salvar itens em CSV sem precisar utilizar bibliotecas externas

In [36]:
with open('pregoes.csv', 'w') as f:
    fieldnames = pregoes[0].keys()
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(pregoes)

criar um DataFrame a partir da variável pregoes

In [25]:
df_pregoes = pd.DataFrame(pregoes)

In [34]:
df_pregoes

,numero_pregao,codigo_uasg,nome_uasg,data_hora_inicio,data_hora_abertura,situacao
0,642021,925013,PMSP - SECRETARIA MUNICIPAL DE EDUCAÇÃO,09/09/2021 08:00,21/09/2021 09:30,Julgamento/Habilitação


In [29]:
from google.colab import files
df_pregoes.to_csv('df_pregoes.csv')
files.download('df_pregoes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>